# Multi-language data anonymization with Microsoft Presidio | Microsoft Presidioによる多言語データの匿名化[![Colabで開く](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/langchain-ai/langchain/blob/master/docs/docs/guides/privacy/presidio_data_anonymization/multi_language.ipynb)> [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/langchain-ai/langchain/blob/master/docs/docs/guides/privacy/presidio_data_anonymization/multi_language.ipynb)## Use case | 使用例データの擬似名化における多言語サポートは、言語構造と文化的文脈の違いにより不可欠です。異なる言語では、個人識別子の形式が異なる場合があります。例えば、名前、場所、日付の構造は言語や地域によって大きく異なることがあります。さらに、非アルファベット文字、アクセント、書き方の方向が擬似名化プロセスに影響を与える可能性があります。多言語サポートがなければ、データが識別可能なまま残るか、誤解されることで、データのプライバシーと正確性が損なわれます。したがって、グローバルなオペレーションに適した効果的で正確な擬似名化を実現するためには、多言語サポートが必要です。> Multi-language support in data pseudonymization is essential due to differences in language structures and cultural contexts. Different languages may have varying formats for personal identifiers. For example, the structure of names, locations and dates can differ greatly between languages and regions. Furthermore, non-alphanumeric characters, accents, and the direction of writing can impact pseudonymization processes. Without multi-language support, data could remain identifiable or be misinterpreted, compromising data privacy and accuracy. Hence, it enables effective and precise pseudonymization suited for global operations.## Overview | 概要Microsoft Presidioにおける個人識別情報（PII）の検出は、いくつかのコンポーネントに依存しています。通常のパターンマッチング（例えば、正規表現を使用する）に加えて、アナライザーはNamed Entity Recognition（NER）のモデルを使用して、以下のようなエンティティを抽出します：> PII detection in Microsoft Presidio relies on several components - in addition to the usual pattern matching (e.g. using regex), the analyser uses a model for Named Entity Recognition (NER) to extract entities such as:* `PERSON`  > `PERSON`* `LOCATION`  > `LOCATION`* `DATE_TIME`  > `DATE_TIME`* `NRP`  > `NRP`* `ORGANIZATION`  > `ORGANIZATION`[\[ソース\]](https://github.com/microsoft/presidio/blob/main/presidio-analyzer/presidio_analyzer/predefined_recognizers/spacy_recognizer.py)> [\[Source\]](https://github.com/microsoft/presidio/blob/main/presidio-analyzer/presidio_analyzer/predefined_recognizers/spacy_recognizer.py)特定の言語での固有表現認識（NER）を処理するために、私たちは`spaCy`ライブラリの独自のモデルを使用しています。このライブラリは、多様な言語とサイズをカバーする広範な選択肢で認識されています。しかし、必要に応じて[Stanza](https://microsoft.github.io/presidio/analyzer/nlp_engines/spacy_stanza/)や[transformers](https://microsoft.github.io/presidio/analyzer/nlp_engines/transformers/)などの代替フレームワークとの統合も可能で、制限はありません。> To handle NER in specific languages, we utilize unique models from the `spaCy` library, recognized for its extensive selection covering multiple languages and sizes. However, it's not restrictive, allowing for integration of alternative frameworks such as [Stanza](https://microsoft.github.io/presidio/analyzer/nlp_engines/spacy_stanza/) or [transformers](https://microsoft.github.io/presidio/analyzer/nlp_engines/transformers/) when necessary.## Quickstart | クイックスタート

In [1]:
# Install necessary packages
# ! pip install langchain langchain-experimental openai presidio-analyzer presidio-anonymizer spacy Faker
# ! python -m spacy download en_core_web_lg

In [1]:
from langchain_experimental.data_anonymizer import PresidioReversibleAnonymizer

anonymizer = PresidioReversibleAnonymizer(
    analyzed_fields=["PERSON"],
)

デフォルトでは、`PresidioAnonymizer`と`PresidioReversibleAnonymizer`は英語のテキストで訓練されたモデルを使用しているため、他の言語に対しても適度にうまく対応します。> By default, `PresidioAnonymizer` and `PresidioReversibleAnonymizer` use a model trained on English texts, so they handle other languages moderately well.例えば、ここではモデルが人物を検出できませんでした：> For example, here the model did not detect the person:

In [10]:
anonymizer.anonymize("Me llamo Sofía")  # "My name is Sofía" in Spanish

'Me llamo Sofía'

また、他の言語の単語を実際のエンティティとして取り扱うこともあります。ここでは、スペイン語で「私」を意味する\*'Yo'*と*Sofía\*が、`PERSON`として分類されています：> They may also take words from another language as actual entities. Here, both the word *'Yo'* (*'I'* in Spanish) and *Sofía* have been classified as `PERSON`:

In [11]:
anonymizer.anonymize("Yo soy Sofía")  # "I am Sofía" in Spanish

'Kari Lopez soy Mary Walker'

他の言語のテキストを匿名化したい場合は、他のモデルをダウンロードして、匿名化設定に追加する必要があります：> If you want to anonymise texts from other languages, you need to download other models and add them to the anonymiser configuration:

In [5]:
# Download the models for the languages you want to use
# ! python -m spacy download en_core_web_md
# ! python -m spacy download es_core_news_md

In [2]:
nlp_config = {
    "nlp_engine_name": "spacy",
    "models": [
        {"lang_code": "en", "model_name": "en_core_web_md"},
        {"lang_code": "es", "model_name": "es_core_news_md"},
    ],
}

そのため、スペイン語の言語モデルを追加しました。また、英語の代替モデルもダウンロードしました - この場合、大きなモデル `en_core_web_lg`（560MB）をより小さいバージョンの `en_core_web_md`（40MB）に置き換えました - サイズは従って14分の1に削減されています！匿名化の速度が重要な場合は、これを検討する価値があるでしょう。> We have therefore added a Spanish language model. Note also that we have downloaded an alternative model for English as well - in this case we have replaced the large model `en_core_web_lg` (560MB) with its smaller version `en_core_web_md` (40MB) - the size is therefore reduced by 14 times! If you care about the speed of anonymisation, it is worth considering it.異なる言語のすべてのモデルは、[spaCyのドキュメント](https://spacy.io/usage/models)で見つけることができます。> All models for the different languages can be found in the [spaCy documentation](https://spacy.io/usage/models).次に、設定を`languages_config`パラメータとしてAnonymiserに渡してください。ご覧の通り、前述の例はどちらも問題なく動作します：> Now pass the configuration as the `languages_config` parameter to Anonymiser. As you can see, both previous examples work flawlessly:

In [3]:
anonymizer = PresidioReversibleAnonymizer(
    analyzed_fields=["PERSON"],
    languages_config=nlp_config,
)

print(
    anonymizer.anonymize("Me llamo Sofía", language="es")
)  # "My name is Sofía" in Spanish
print(anonymizer.anonymize("Yo soy Sofía", language="es"))  # "I am Sofía" in Spanish

Me llamo Christopher Smith
Yo soy Joseph Jenkins


デフォルトでは、テキストを匿名化する際には設定で最初に指定された言語が使用されます（この場合、英語です）：> By default, the language indicated first in the configuration will be used when anonymising text (in this case English):

In [14]:
print(anonymizer.anonymize("My name is John"))

My name is Shawna Bennett


## Usage with other frameworks | 他のフレームワークとの使用方法### Language detection | 言語検出提示されたアプローチの欠点の一つは、入力テキストの**言語**を直接指定しなければならないことです。しかし、その対策として*言語検出*ライブラリがあります。> One of the drawbacks of the presented approach is that we have to pass the **language** of the input text directly. However, there is a remedy for that - *language detection* libraries.以下のフレームワークのいずれかを使用することをお勧めします：> We recommend using one of the following frameworks:* fasttext（推奨）  > fasttext (recommended)* langdetect  > langdetect私たちの経験によると、*fasttext* は少し優れていると思われますが、ご自身のユースケースで検証することをお勧めします。> From our experience *fasttext* performs a bit better, but you should verify it on your use case.

In [ ]:
# Install necessary packages
# ! pip install fasttext langdetect

### langdetect | langdetect

In [7]:
import langdetect
from langchain.schema import runnable


def detect_language(text: str) -> dict:
    language = langdetect.detect(text)
    print(language)
    return {"text": text, "language": language}


chain = runnable.RunnableLambda(detect_language) | (
    lambda x: anonymizer.anonymize(x["text"], language=x["language"])
)

In [15]:
chain.invoke("Me llamo Sofía")

es


'Me llamo Michael Perez III'

In [12]:
chain.invoke("My name is John Doe")

en


'My name is Ronald Bennett'

### fasttext | fasttext

まず、https://dl.fbaipublicfiles.com/fasttext/supervised-models/lid.176.ftz からfasttextモデルをダウンロードする必要があります。> You need to download the fasttext model first from https://dl.fbaipublicfiles.com/fasttext/supervised-models/lid.176.ftz

In [18]:
import fasttext

model = fasttext.load_model("lid.176.ftz")


def detect_language(text: str) -> dict:
    language = model.predict(text)[0][0].replace("__label__", "")
    print(language)
    return {"text": text, "language": language}


chain = runnable.RunnableLambda(detect_language) | (
    lambda x: anonymizer.anonymize(x["text"], language=x["language"])
)

In [21]:
chain.invoke("Yo soy Sofía")

es


'Yo soy Angela Werner'

In [20]:
chain.invoke("My name is John Doe")

en


'My name is Carlos Newton'

この方法では、関連する言語に対応するエンジンでモデルを初期化するだけで済み、ツールの使用は完全に自動化されています。> This way you only need to initialize the model with the engines corresponding to the relevant languages, but using the tool is fully automated.

## Advanced usage | 高度な使い方### Custom labels in NER model | NERモデルのカスタムラベル

たとえば、spaCyモデルには、デフォルトでMicrosoft Presidioがサポートするクラス名とは異なるものがあるかもしれません。ポーランド語を例にとると：> It may be that the spaCy model has different class names than those supported by the Microsoft Presidio by default. Take Polish, for example:

In [9]:
# ! python -m spacy download pl_core_news_md

import spacy

nlp = spacy.load("pl_core_news_md")
doc = nlp("Nazywam się Wiktoria")  # "My name is Wiktoria" in Polish

for ent in doc.ents:
    print(
        f"Text: {ent.text}, Start: {ent.start_char}, End: {ent.end_char}, Label: {ent.label_}"
    )

Text: Wiktoria, Start: 12, End: 20, Label: persName


名前 *Victoria* は `persName` として分類されましたが、これは Microsoft Presidio において実装されているデフォルトのクラス名 `PERSON`/`PER` とは対応していません（[SpacyRecognizerの実装](https://github.com/microsoft/presidio/blob/main/presidio-analyzer/presidio_analyzer/predefined_recognizers/spacy_recognizer.py)で `CHECK_LABEL_GROUPS` を探してください）。> The name *Victoria* was classified as `persName`, which does not correspond to the default class names `PERSON`/`PER` implemented in Microsoft Presidio (look for `CHECK_LABEL_GROUPS` in [SpacyRecognizer implementation](https://github.com/microsoft/presidio/blob/main/presidio-analyzer/presidio_analyzer/predefined_recognizers/spacy_recognizer.py)).spaCyモデルのカスタムラベル（自分でトレーニングしたものを含む）についての詳細は、[このスレッド](https://github.com/microsoft/presidio/issues/851)で確認できます。> You can find out more about custom labels in spaCy models (including your own, trained ones) in [this thread](https://github.com/microsoft/presidio/issues/851).そのため、私たちの文は匿名化されません：> That's why our sentence will not be anonymized:

In [10]:
nlp_config = {
    "nlp_engine_name": "spacy",
    "models": [
        {"lang_code": "en", "model_name": "en_core_web_md"},
        {"lang_code": "es", "model_name": "es_core_news_md"},
        {"lang_code": "pl", "model_name": "pl_core_news_md"},
    ],
}

anonymizer = PresidioReversibleAnonymizer(
    analyzed_fields=["PERSON", "LOCATION", "DATE_TIME"],
    languages_config=nlp_config,
)

print(
    anonymizer.anonymize("Nazywam się Wiktoria", language="pl")
)  # "My name is Wiktoria" in Polish

Nazywam się Wiktoria


これに対処するために、独自のクラスマッピングを持つ`SpacyRecognizer`を作成し、それを匿名化ツールに追加してください：> To address this, create your own `SpacyRecognizer` with your own class mapping and add it to the anonymizer:

In [11]:
from presidio_analyzer.predefined_recognizers import SpacyRecognizer

polish_check_label_groups = [
    ({"LOCATION"}, {"placeName", "geogName"}),
    ({"PERSON"}, {"persName"}),
    ({"DATE_TIME"}, {"date", "time"}),
]

spacy_recognizer = SpacyRecognizer(
    supported_language="pl",
    check_label_groups=polish_check_label_groups,
)

anonymizer.add_recognizer(spacy_recognizer)

これで全てがスムーズに機能します：> Now everything works smoothly:

In [12]:
print(
    anonymizer.anonymize("Nazywam się Wiktoria", language="pl")
)  # "My name is Wiktoria" in Polish

Nazywam się Morgan Walters


もっと複雑な例を試してみましょう：> Let's try on more complex example:

In [13]:
print(
    anonymizer.anonymize(
        "Nazywam się Wiktoria. Płock to moje miasto rodzinne. Urodziłam się dnia 6 kwietnia 2001 roku",
        language="pl",
    )
)  # "My name is Wiktoria. Płock is my home town. I was born on 6 April 2001" in Polish

Nazywam się Ernest Liu. New Taylorburgh to moje miasto rodzinne. Urodziłam się 1987-01-19


ご覧の通り、クラスマッピングのおかげで、匿名化ツールはさまざまなタイプのエンティティに対応できます。> As you can see, thanks to class mapping, the anonymiser can cope with different types of entities.

### Custom language-specific operators | 言語固有のカスタムオペレーター上記の例では、文章は正しく匿名化されていますが、偽のデータはポーランド語に全く適していません。したがって、この問題を解決するためにカスタムオペレーターを追加できます：> In the example above, the sentence has been anonymised correctly, but the fake data does not fit the Polish language at all. Custom operators can therefore be added, which will resolve the issue:

In [14]:
from faker import Faker
from presidio_anonymizer.entities import OperatorConfig

fake = Faker(locale="pl_PL")  # Setting faker to provide Polish data

new_operators = {
    "PERSON": OperatorConfig("custom", {"lambda": lambda _: fake.first_name_female()}),
    "LOCATION": OperatorConfig("custom", {"lambda": lambda _: fake.city()}),
}

anonymizer.add_operators(new_operators)

In [15]:
print(
    anonymizer.anonymize(
        "Nazywam się Wiktoria. Płock to moje miasto rodzinne. Urodziłam się dnia 6 kwietnia 2001 roku",
        language="pl",
    )
)  # "My name is Wiktoria. Płock is my home town. I was born on 6 April 2001" in Polish

Nazywam się Marianna. Szczecin to moje miasto rodzinne. Urodziłam się 1976-11-16


### Limitations | 制限事項覚えておいてください - 結果は、あなたの認識器とNERモデルの質によって決まります！> Remember - results are as good as your recognizers and as your NER models!以下の例を見てください。スペイン語の小さいモデル（12MB）をダウンロードしましたが、中サイズのバージョン（40MB）ほどの性能は発揮できませんでした：> Look at the example below - we downloaded the small model for Spanish (12MB) and it no longer performs as well as the medium version (40MB):

In [16]:
# ! python -m spacy download es_core_news_sm

for model in ["es_core_news_sm", "es_core_news_md"]:
    nlp_config = {
        "nlp_engine_name": "spacy",
        "models": [
            {"lang_code": "es", "model_name": model},
        ],
    }

    anonymizer = PresidioReversibleAnonymizer(
        analyzed_fields=["PERSON"],
        languages_config=nlp_config,
    )

    print(
        f"Model: {model}. Result: {anonymizer.anonymize('Me llamo Sofía', language='es')}"
    )

Model: es_core_news_sm. Result: Me llamo Sofía
Model: es_core_news_md. Result: Me llamo Lawrence Davis


多くの場合、spaCyの大規模なモデルでさえ十分ではないことがあります - 名前付きエンティティを検出するための、トランスフォーマーに基づく他の、より複雑で優れた方法が既に存在しています。これについての詳細は[こちら](https://microsoft.github.io/presidio/analyzer/nlp_engines/transformers/)で読むことができます。> In many cases, even the larger models from spaCy will not be sufficient - there are already other, more complex and better methods of detecting named entities, based on transformers. You can read more about this [here](https://microsoft.github.io/presidio/analyzer/nlp_engines/transformers/).